In [175]:
import itertools
import sys, string, os
home_dir = '/Users/michaellomnitz/Documents/eSTARlight/x_sections/'
particle_id = [ 113, 333, 443011, 444011, 553011]
#particle_id = [ 444011, 553011]
particle_name = ['phi', 'rho', 'J_psi', 'Psi_2s', 'Upsilon_1s']
#particle_name = [ 'Psi_2s', 'Upsilon_1s']
unit_decode = { 'mb.':-3, 'microb.':-6, 'nanob.':-9, 'picob.':-12, 'femtob.':-15}
unit_latex = { 'mb.':'mb', 'microb.':'$\mu$b', 'nanob.':'nb', 'picob.':'pb', 'femtob.':'fb'}
event_unit = { 3:'K', 6:'M', 9:'G'}
n_events_from_units = {'mb.':'T', 'microb.':'G', 'nanob.':'M', 'picob.':'K', 'femtob.':''} #considering 1 fb^-1

In [164]:
from math import log10, floor
def round_sig(x, sig=2):
    return round(x, sig-int(floor(log10(abs(x))))-1)

def make_input_file(prod_no, template_loc):
    #load template
    with open(template_loc,'r') as f:
        data = f.readlines()
    # --  Set prodiction id
    data[29] = 'PROD_PID = '+str(particle_id[prod_no])+'   #Channel of interest; this is '+str(particle_name[prod_no])+'\n'
    with open(home_dir+'STARlight/slight.in','w') as f:
        f.writelines(data)

def run_accel_study(accel_name, prod_mech, pwd_to_template):
    print(len(particle_id),' particles in study')
    for idx,part in enumerate(particle_id):
        make_input_file(idx,pwd_to_template)
        os.chdir( home_dir+'STARlight' )
        os.system( home_dir+'STARlight/e_starlight > log' )
        os.rename(home_dir+'STARlight/log',home_dir+accel_name+'/'+prod_mech+'/x_section_'+particle_name[idx]+'.txt')
        print(idx,'/',len(particle_id),' done:',particle_name[idx])
    
def decode_string( line ):
    space_pos = [pos for pos,char in enumerate(line) if char == ' ']
    n_space = len(space_pos)
    num = float(line[space_pos[n_space-2]:space_pos[n_space-1]])
    num = round_sig(num,2)
    units = line[space_pos[n_space-1]+1:-1]
    return num,units
    
def find_x_section(accel_name, prod_mech, part_id):
    file_loc = home_dir+accel_name+'/'+prod_mech+'/x_section_'+particle_name[part_id]+'.txt'
    with open(file_loc,'r') as f:
        data = f.readlines() 
    x_sec, units = decode_string(data[46])
    return x_sec, units

def make_table_line(collision_system, prod_mech, target_A):
    particle_row = ''
    x_sec_row = ' & '
    rate_row = ''
    for idx, a in enumerate(particle_name):
        x_sec, units = find_x_section(collision_system, prod_mech, idx)
        n_events = round(10*x_sec/target_A,2)
        particle_row+=a+' & '
        x_sec_row+='$'+str(x_sec)+'$ '+unit_latex[units]+' & '
        rate_row+= str(n_events)+n_events_from_units[units]
        if idx != len(particle_name)-1:
            rate_row+=' & '
        else:
            rate_row+='\\\\'
    return x_sec_row+rate_row

In [168]:
run_accel_study('JLAB_eA','EP',home_dir+'slight_template_JLAB_eA.in')

5  particles in study
0 / 5  done: phi
1 / 5  done: rho
2 / 5  done: J_psi
3 / 5  done: Psi_2s
4 / 5  done: Upsilon_1s


In [182]:
print(make_table_line('JLAB_eA','PP', 208))

 & $350.0$ $\mu$b & $14.0$ $\mu$b & $67.0$ nb & $18.0$ nb & $660.0$ pb & 16.83G & 0.67G & 3.22M & 0.87M & 31.73K\\


In [162]:
_prod_rows = { 'eRHIC':['eRHIC - $ep$',1], 'eRHIC_eA':['eRHIC - $eA$',197],
               'JLEIC':['JLEIC - $ep$',1], 'JLEIC_eA':['JLEIC - $eA$',208],
               'LHeC' :['LHeC - $ep$',1],  'LHEC_eA' :['LHeC - $eA$',208],
               'HERA':['HERA - $ep$',1]
             }
def make_full_table( prod_mech ):
    for idx,key in enumerate(_prod_rows):
        #print(idx,_prod_rows[key][0])
        to_print = _prod_rows[key][0] + make_table_line(key,prod_mech,_prod_rows[key][1])
        print( to_print )
        if (idx+1)%2 == 0:
            print('\hline')
make_full_table('EP')

eRHIC - $ep$ & $19.0$ nb & $2.0$ nb & $590.0$ pb & $120.0$ pb & $3.3$ pb & 190.0M & 20.0M & 5900.0K & 1200.0K & 33.0K\\
eRHIC - $eA$ & $5.8$ $\mu$b & $770.0$ nb & $200.0$ nb & $47.0$ nb & $1.2$ nb & 0.29G & 39.09M & 10.15M & 2.39M & 0.06M\\
\hline
JLEIC - $ep$ & $15.0$ nb & $1.5$ nb & $290.0$ pb & $60.0$ pb & $1.7$ pb & 150.0M & 15.0M & 2900.0K & 600.0K & 17.0K\\
JLEIC - $eA$ & $5.0$ $\mu$b & $620.0$ nb & $100.0$ nb & $24.0$ nb & $710.0$ pb & 0.24G & 29.81M & 4.81M & 1.15M & 34.13K\\
\hline
LHeC - $ep$ & $24.0$ nb & $2.7$ nb & $1.1$ nb & $240.0$ pb & $6.4$ pb & 240.0M & 27.0M & 11.0M & 2400.0K & 64.0K\\
LHeC - $eA$ & $10.0$ $\mu$b & $1.5$ $\mu$b & $1.0$ $\mu$b & $260.0$ nb & $7.9$ nb & 0.48G & 0.07G & 0.05G & 12.5M & 0.38M\\
\hline
HERA - $ep$ & $24.0$ nb & $2.7$ nb & $1.1$ nb & $240.0$ pb & $6.4$ pb & 240.0M & 27.0M & 11.0M & 2400.0K & 64.0K\\


In [144]:


print(round_sig(1236,3))

1240
